In [39]:
import os.path
import numpy as np

In [40]:
path_prefix = '../CS640_Project_dataset/'
dir_prefix = 'scene_'
obj_suffix = '.JPG'
file_scene = 'scene.jpg'
file_xml = 'scene.xml'

ikea_scenes = [
    'bathroom', 'bedroom', 'childrenroom', 
    'hallway', 'homeoffice', 'kitchen', 
    'laundry', 'livingroom', 'outdoor']

In [89]:
def scan_num_obj(path):
    i = 0
    if not os.path.exists(path + '/' + file_xml):
        return i
    while True:
        fname = path + '/' + str(i+1) + obj_suffix
        if os.path.exists(fname):
            i += 1
        else:
            break
    return i

def scan_scene(path_to_scene):
    count = []
    i = 1
    while True:
        path_name = path_to_scene + '/' + dir_prefix + str(i)
        i += 1 
        if os.path.exists(path_name):
            count.append(scan_num_obj(path_name))
        else:
            break
    return count

def scan_all(path_to_data):
    output = []
    for x in ikea_scenes:
        output.append(scan_scene(path_to_data + '/' + x))
    return output

dataset_structure = scan_all(path_prefix)

In [101]:
def backtrack_path(target_num, path_to_data):
    scene_loc = 0
    scene_num = 0
    found = False
    for x in dataset_structure:
        for y in x:
            if (target_num <= y):
                found = True
                break
            else: 
                target_num -= y
                scene_num += 1
        if found:
            break
        scene_loc += 1
    return (scene_loc, scene_num + 1, target_num)

In [102]:
backtrack_path(35, path_prefix)

(0, 4, 5)

In [ ]:

def generate_target():
    return None